In [10]:
import pandas as pd
import datetime

file_names = ['data/potholes_1.csv', 'data/potholes_2.csv']

potholes = pd.concat((pd.read_csv(i) for i in file_names))


potholes = potholes.iloc[:,
    [0, 1, 2, 5, 6, 9, 10, 11, 12, 13, 14, 16, 17, 18
    ]
]
potholes = potholes.dropna(subset = ["ZIP", "Community Area"])
potholes['Community Area'] = potholes['Community Area'].astype(int)

potholes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558055 entries, 1 to 310476
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CREATION DATE       558055 non-null  object 
 1   STATUS              558055 non-null  object 
 2   COMPLETION DATE     550786 non-null  object 
 3   CURRENT ACTIVITY    391921 non-null  object 
 4   MOST RECENT ACTION  390310 non-null  object 
 5   ZIP                 558055 non-null  float64
 6   X COORDINATE        557921 non-null  float64
 7   Y COORDINATE        557921 non-null  float64
 8   Ward                558054 non-null  float64
 9   Police District     558054 non-null  float64
 10  Community Area      558055 non-null  int64  
 11  LATITUDE            557271 non-null  float64
 12  LONGITUDE           557271 non-null  float64
 13  LOCATION            557271 non-null  object 
dtypes: float64(7), int64(1), object(6)
memory usage: 63.9+ MB


pandas.core.series.Series

In [11]:
# Convert strings to times
potholes["CREATION DATE"] = datetime.strptime(potholes["CREATION DATE"])

str

In [2]:
census_data = pd.read_csv('data/chicago_census_data.csv')
census_data = census_data.iloc[:, [0,7,8]].dropna()

In [3]:
potholes_c = potholes.groupby(["Community Area"])['CREATION DATE'].count()
pd_holes = potholes_c.to_frame(name='Count').reset_index()

In [6]:
census_data.rename(columns = {'Community Area Number': 'Community Area'}, inplace = True)
census_data['Community Area'] = census_data['Community Area'].astype(int)
working_data = pd.merge(census_data, pd_holes, on = 'Community Area').sort_values("Count", ascending = False)
working_data

,Community Area,PER CAPITA INCOME,HARDSHIP INDEX,Count
24,25,15957,73.0,19454
23,24,43198,10.0,17585
21,22,31908,23.0,15507
1,2,23040,46.0,15426
27,28,44689,15.0,14397
...,...,...,...,...
38,39,35911,26.0,1794
53,54,8201,98.0,1580
36,37,10432,97.0,1051
46,47,12515,79.0,856


In [ ]:
#Completion Time

